# Comparison of Intel Optimized TensorFlow vs Default Tensorflow



|                               | Intel Optimized TensorFlow | Default TensorFlow |
|-------------------------------|----------------------------|--------------------|
| Normalized Data (10 Epochs)   | 10.90 seconds              | 30.77 seconds      |
| Normalized Data (20 Epochs)   | 22.87 seconds              | 66.84 seconds      |
|                               |                            |                    |
| Unnormalized Data (10 Epochs) | 50.57 seconds              | 73.14 seconds      |
| Unnormalized Data (20 Epochs) | 101.56 seconds             | 142.69 seconds     |

# Import Libraries

### When Intel® Extension for TensorFlow is used, it informs us at the output

In [ ]:
import time
import pathlib
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D


2023-08-07 13:47:04.699283: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-07 13:47:04.738688: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-07 13:47:04.740150: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 13:47:06.697674: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-07 13:47:08.405630: I itex/core/wrapper/itex_cpu_wrapper.cc:42] Intel Extension for Tensorflow* AVX512 CPU backend is loaded.
2023-08-07 13:47:08.447965: W itex/core/ops/op_init.cc:58] Op: _QuantizedMaxPool3D is already registered in Tensorflow


### When Intel® Extension for TensorFlow is not used, the output shown like this

In [ ]:
import time
import pathlib
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D


2023-08-07 14:00:52.679834: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Creat Dataset

In [ ]:
num_classes = 5
batch_size = 32
img_height = 160
img_width = 160

train_dir = pathlib.Path("/home/u198181/0image_classification/DATA/train/")
test_dir = pathlib.Path("/home/u198181/0image_classification/DATA/testing/")

In [ ]:
image_count_train = len(list(train_dir.glob('*/*.jpg')))
print("Number of Train Images : ",image_count_train)
image_count_test = len(list(test_dir.glob('*/*.jpg')))
print("Number of Test Images  : ",image_count_test)

Number of Train Images :  440
Number of Test Images  :  183


In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  shuffle=True,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 440 files belonging to 5 classes.


2023-08-09 00:00:24.866305: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  shuffle=True,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 440 files belonging to 5 classes.
Using 88 files for validation.


In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 185 files belonging to 5 classes.


In [ ]:
class_names = train_dataset.class_names
print(class_names)

['Acne', 'Actinic Keratosis', 'Basal Cell Carcinoma', 'Eczemaa', 'Rosacea']


In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.5, 0.2),
    layers.RandomContrast(0.3, seed=123),
    layers.RandomTranslation(0.2, 0.4),
    layers.RandomFlip(mode="horizontal_and_vertical", seed=412)
  ]
)

In [ ]:
aug_train_dataset = train_dataset.map(
  lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
normalization_layer = layers.Rescaling(1./255)

#normalization_layer = layers.Rescaling(1./127.5, offset=-1) # MobileNet should have this format

In [ ]:
normalized_train_dataset = aug_train_dataset.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_train_dataset))
first_image = image_batch[0]

# Notice the pixel values are now in `[-1,1]`.
print(np.min(first_image), np.max(first_image))

2023-08-08 23:58:14.208617: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-08 23:58:14.298041: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-08 23:58:14.303936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


0.085818656 0.8278934


In [ ]:
normalized_train_dataset = aug_train_dataset.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_train_dataset))
first_image = image_batch[0]

# Notice the pixel values are now in `[-1,1]`.
print(np.min(first_image), np.max(first_image))

2023-08-09 00:00:42.187327: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_19' with dtype resource
	 [[{{node Placeholder/_19}}]]
2023-08-09 00:00:42.188370: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [440]
	 [[{{node Placeholder/_0}}]]


0.17204624 0.9189608


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

normalized_train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Model

In [ ]:
num_classes = 5

model = Sequential([
  #data_augmentation, # Since these options applied to data in the previous step,
  #layers.Rescaling(1./255), # There is no need to implement them in model structure
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])



In [ ]:

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# Training with Intel Extension for TensorFlow

## Normalized Dataset

### 10 Epochs

In [ ]:
totaltime = 0
start = time.time()

history = model.fit(normalized_train_dataset,
                    validation_data=validation_dataset,
                    epochs=10, verbose=1,
                   )

end = time.time()

totaltime = end - start

print("Training was completed in ",totaltime," seconds.")

Epoch 1/10
14/14 [==============================] - 1s 77ms/step - loss: 0.2179 - accuracy: 0.9341 - val_loss: 0.1270 - val_accuracy: 0.9432
Epoch 2/10
14/14 [==============================] - 1s 77ms/step - loss: 0.1974 - accuracy: 0.9364 - val_loss: 0.1437 - val_accuracy: 0.9432
Epoch 3/10
14/14 [==============================] - 1s 77ms/step - loss: 0.1755 - accuracy: 0.9341 - val_loss: 0.0796 - val_accuracy: 0.9886
Epoch 4/10
14/14 [==============================] - 1s 77ms/step - loss: 0.1374 - accuracy: 0.9636 - val_loss: 0.0828 - val_accuracy: 0.9659
Epoch 5/10
14/14 [==============================] - 1s 78ms/step - loss: 0.1646 - accuracy: 0.9500 - val_loss: 0.1440 - val_accuracy: 0.9545
Epoch 6/10
14/14 [==============================] - 1s 79ms/step - loss: 0.2010 - accuracy: 0.9386 - val_loss: 0.1429 - val_accuracy: 0.9545
Epoch 7/10
14/14 [==============================] - 1s 79ms/step - loss: 0.1713 - accuracy: 0.9523 - val_loss: 0.1045 - val_accuracy: 0.9545
Epoch 8/10
14

### 20 Epochs

In [ ]:
totaltime = 0
start = time.time()

history = model.fit(normalized_train_dataset,
                    validation_data=validation_dataset,
                    epochs=20, verbose=1,
                   )

end = time.time()

totaltime = end - start

print("Training was completed in ",totaltime," seconds.")

Epoch 1/20


2023-08-07 13:45:23.140004: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-07 13:45:23.281298: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


14/14 [==============================] - 2s 89ms/step - loss: 79.4564 - accuracy: 0.2068 - val_loss: 2.2165 - val_accuracy: 0.1591
Epoch 2/20


2023-08-07 13:45:24.641889: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-07 13:45:24.696163: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


14/14 [==============================] - 1s 71ms/step - loss: 1.5555 - accuracy: 0.3477 - val_loss: 1.2147 - val_accuracy: 0.5568
Epoch 3/20
14/14 [==============================] - 1s 80ms/step - loss: 1.1934 - accuracy: 0.5523 - val_loss: 1.0099 - val_accuracy: 0.7045
Epoch 4/20
14/14 [==============================] - 1s 73ms/step - loss: 0.9866 - accuracy: 0.6523 - val_loss: 0.8281 - val_accuracy: 0.6818
Epoch 5/20
14/14 [==============================] - 1s 75ms/step - loss: 0.7971 - accuracy: 0.6909 - val_loss: 0.6531 - val_accuracy: 0.7273
Epoch 6/20
14/14 [==============================] - 1s 76ms/step - loss: 0.6012 - accuracy: 0.7727 - val_loss: 0.4584 - val_accuracy: 0.8409
Epoch 7/20
14/14 [==============================] - 1s 70ms/step - loss: 0.4867 - accuracy: 0.8205 - val_loss: 0.4290 - val_accuracy: 0.8750
Epoch 8/20
14/14 [==============================] - 1s 76ms/step - loss: 0.5126 - accuracy: 0.8682 - val_loss: 0.2999 - val_accuracy: 0.9091
Epoch 9/20
14/14 [======

## Unnormalized Data

### 10 Epochs

In [ ]:
totaltime = 0
start = time.time()

history = model.fit(aug_train_dataset,
                    validation_data=validation_dataset,
                    epochs=10, verbose=1,
                    )
end = time.time()

totaltime = end - start

print("Training was completed in ",totaltime," seconds.")

2023-08-07 13:54:31.593122: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-07 13:54:31.702851: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


Epoch 1/10


2023-08-07 13:54:31.809235: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


14/14 [==============================] - 5s 299ms/step - loss: 3.2534 - accuracy: 0.2818 - val_loss: 1.2197 - val_accuracy: 0.4205


2023-08-07 13:54:36.329877: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


...

14/14 [==============================] - 4s 277ms/step - loss: 1.5286 - accuracy: 0.2727 - val_loss: 1.3658 - val_accuracy: 0.4545


2023-08-07 13:55:16.949856: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


Epoch 10/10


2023-08-07 13:55:17.828895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


14/14 [==============================] - 4s 281ms/step - loss: 1.5218 - accuracy: 0.2932 - val_loss: 1.3767 - val_accuracy: 0.4773
Training was completed in  50.576711654663086  seconds.


### 20 Epochs

In [ ]:
totaltime = 0
start = time.time()

history = model.fit(aug_train_dataset,
                    validation_data=validation_dataset,
                    epochs=20, verbose=1,
                    )
end = time.time()

totaltime = end - start

print("Training was completed in ",totaltime," seconds.")

Epoch 1/20


2023-08-07 13:58:18.871503: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-07 13:58:18.974646: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-07 13:58:19.068751: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


14/14 [==============================] - 4s 282ms/step - loss: 1.4514 - accuracy: 0.3318 - val_loss: 1.3810 - val_accuracy: 0.3523


2023-08-07 13:58:23.328291: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


Epoch 2/20


...

14/14 [==============================] - 4s 286ms/step - loss: 1.4628 - accuracy: 0.3364 - val_loss: 1.3987 - val_accuracy: 0.3750


2023-08-07 13:59:55.413351: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


Epoch 20/20


2023-08-07 13:59:56.224299: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


14/14 [==============================] - 4s 279ms/step - loss: 1.4358 - accuracy: 0.3932 - val_loss: 1.4421 - val_accuracy: 0.3523
Training was completed in  101.56522107124329  seconds.


# Default Tensorflow

## Normalized Dataset

### 10 Epochs

In [ ]:
totaltime = 0
start = time.time()

history = model.fit(normalized_train_dataset,
                    validation_data=validation_dataset,
                    epochs=10, verbose=1,
                    )
end = time.time()

totaltime = end - start

print("Training was completed in ",totaltime," seconds.")

Epoch 1/10


2023-08-07 13:38:29.121842: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [440]
	 [[{{node Placeholder/_4}}]]
2023-08-07 13:38:29.122693: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [440]
	 [[{{node Placeholder/_4}}]]


14/14 [==============================] - ETA: 0s - loss: 120.8987 - accuracy: 0.2477

2023-08-07 13:38:33.453439: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [88]
	 [[{{node Placeholder/_0}}]]
2023-08-07 13:38:33.453987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [88]
	 [[{{node Placeholder/_4}}]]


14/14 [==============================] - 5s 207ms/step - loss: 120.8987 - accuracy: 0.2477 - val_loss: 2.3418 - val_accuracy: 0.3182
Epoch 2/10
14/14 [==============================] - 3s 195ms/step - loss: 1.6044 - accuracy: 0.3114 - val_loss: 1.3214 - val_accuracy: 0.4091
Epoch 3/10
14/14 [==============================] - 3s 196ms/step - loss: 1.3189 - accuracy: 0.4409 - val_loss: 1.1904 - val_accuracy: 0.5000
Epoch 4/10
14/14 [==============================] - 3s 208ms/step - loss: 1.0821 - accuracy: 0.5455 - val_loss: 0.8605 - val_accuracy: 0.6932
Epoch 5/10
14/14 [==============================] - 3s 204ms/step - loss: 0.8649 - accuracy: 0.6455 - val_loss: 0.6350 - val_accuracy: 0.7045
Epoch 6/10
14/14 [==============================] - 3s 186ms/step - loss: 0.6784 - accuracy: 0.7364 - val_loss: 0.4920 - val_accuracy: 0.8636
Epoch 7/10
14/14 [==============================] - 3s 192ms/step - loss: 0.4828 - accuracy: 0.8705 - val_loss: 0.3388 - val_accuracy: 0.8864
Epoch 8/10
14/1

### 20 Epochs

In [ ]:
totaltime = 0
start = time.time()

history = model.fit(normalized_train_dataset,
                    validation_data=validation_dataset,
                    epochs=20, verbose=1,
                    )
end = time.time()

totaltime = end - start

print("Training was completed in ",totaltime," seconds.")

Epoch 1/20


2023-08-09 00:01:44.547366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [440]
	 [[{{node Placeholder/_4}}]]
2023-08-09 00:01:44.548281: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [440]
	 [[{{node Placeholder/_0}}]]


14/14 [==============================] - ETA: 0s - loss: 97.9396 - accuracy: 0.2205 

2023-08-09 00:01:48.868710: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [88]
	 [[{{node Placeholder/_0}}]]
2023-08-09 00:01:48.869219: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [88]
	 [[{{node Placeholder/_4}}]]


14/14 [==============================] - 5s 189ms/step - loss: 97.9396 - accuracy: 0.2205 - val_loss: 1.3874 - val_accuracy: 0.3750
Epoch 2/20
14/14 [==============================] - 3s 183ms/step - loss: 1.4050 - accuracy: 0.3432 - val_loss: 1.2019 - val_accuracy: 0.5114
Epoch 3/20
14/14 [==============================] - 3s 194ms/step - loss: 1.2002 - accuracy: 0.5364 - val_loss: 0.9293 - val_accuracy: 0.6477
Epoch 4/20
14/14 [==============================] - 3s 198ms/step - loss: 0.9476 - accuracy: 0.6250 - val_loss: 0.7349 - val_accuracy: 0.7045
Epoch 5/20
14/14 [==============================] - 3s 190ms/step - loss: 0.7609 - accuracy: 0.7250 - val_loss: 0.4493 - val_accuracy: 0.8636
Epoch 6/20
14/14 [==============================] - 3s 191ms/step - loss: 0.5647 - accuracy: 0.7864 - val_loss: 0.3073 - val_accuracy: 0.9318
Epoch 7/20
14/14 [==============================] - 3s 191ms/step - loss: 0.4824 - accuracy: 0.8455 - val_loss: 0.3666 - val_accuracy: 0.9091
Epoch 8/20
14/14

## Unnormalized Dataset

### 10 Epochs

In [ ]:
totaltime = 0
start = time.time()

history = model.fit(aug_train_dataset,
                    validation_data=validation_dataset,
                    epochs=10, verbose=1,
                    )
end = time.time()

totaltime = end - start

print("Training was completed in ",totaltime," seconds.")

2023-08-07 14:02:23.044939: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_22' with dtype resource
	 [[{{node Placeholder/_22}}]]
2023-08-07 14:02:23.045404: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_22' with dtype resource
	 [[{{node Placeholder/_22}}]]


Epoch 1/10
14/14 [==============================] - ETA: 0s - loss: 228.3792 - accuracy: 0.1932

2023-08-07 14:02:30.829476: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [88]
	 [[{{node Placeholder/_4}}]]
2023-08-07 14:02:30.830167: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [88]
	 [[{{node Placeholder/_4}}]]


14/14 [==============================] - 8s 488ms/step - loss: 228.3792 - accuracy: 0.1932 - val_loss: 1.7724 - val_accuracy: 0.2273
Epoch 2/10
14/14 [==============================] - 7s 471ms/step - loss: 1.6324 - accuracy: 0.2568 - val_loss: 1.5307 - val_accuracy: 0.2500
Epoch 3/10
14/14 [==============================] - 7s 478ms/step - loss: 1.5949 - accuracy: 0.2182 - val_loss: 1.5069 - val_accuracy: 0.3182
Epoch 4/10
14/14 [==============================] - 7s 462ms/step - loss: 1.5604 - accuracy: 0.2727 - val_loss: 1.4670 - val_accuracy: 0.3636
Epoch 5/10
14/14 [==============================] - 7s 456ms/step - loss: 1.5762 - accuracy: 0.2682 - val_loss: 1.4918 - val_accuracy: 0.2727
Epoch 6/10
14/14 [==============================] - 7s 465ms/step - loss: 1.5378 - accuracy: 0.2955 - val_loss: 1.4302 - val_accuracy: 0.3295
Epoch 7/10
14/14 [==============================] - 7s 461ms/step - loss: 1.5986 - accuracy: 0.2727 - val_loss: 1.4510 - val_accuracy: 0.3182
Epoch 8/10
14/1

### 20 Epochs

In [ ]:
totaltime = 0
start = time.time()

history = model.fit(aug_train_dataset,
                    validation_data=validation_dataset,
                    epochs=20, verbose=1,
                    )
end = time.time()

totaltime = end - start

print("Training was completed in ",totaltime," seconds.")

Epoch 1/20
14/14 [==============================] - 6s 446ms/step - loss: 1.5266 - accuracy: 0.3159 - val_loss: 1.4958 - val_accuracy: 0.3182
Epoch 2/20
14/14 [==============================] - 7s 472ms/step - loss: 1.4806 - accuracy: 0.3409 - val_loss: 1.5098 - val_accuracy: 0.3295
Epoch 3/20
14/14 [==============================] - 6s 453ms/step - loss: 1.5141 - accuracy: 0.3136 - val_loss: 1.5193 - val_accuracy: 0.2955
Epoch 4/20
14/14 [==============================] - 7s 456ms/step - loss: 1.5280 - accuracy: 0.3182 - val_loss: 1.5566 - val_accuracy: 0.2614
Epoch 5/20
14/14 [==============================] - 7s 466ms/step - loss: 1.5440 - accuracy: 0.2955 - val_loss: 1.4806 - val_accuracy: 0.2955
Epoch 6/20
14/14 [==============================] - 7s 464ms/step - loss: 1.4967 - accuracy: 0.3409 - val_loss: 1.5071 - val_accuracy: 0.3068
Epoch 7/20
14/14 [==============================] - 7s 462ms/step - loss: 1.4748 - accuracy: 0.3295 - val_loss: 1.5104 - val_accuracy: 0.3068
Epoch 